<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/TensorFlow/Advanced/ProtT5-XL-BFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Extracting protein sequences' features using ProtT5-XL-BFD pretrained-model</h3>


**1. Load necessry libraries including huggingface transformers**

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.1MB 9.4MB/s 
     |████████████████████████████████| 1.1MB 35.8MB/s 
     |████████████████████████████████| 3.0MB 62.7MB/s 
     |████████████████████████████████| 890kB 63.0MB/s 


In [2]:
from transformers import TFT5Model, T5Tokenizer,T5Config
import numpy as np
import re
import gc

<b>2. Load the vocabulary and ProtT5-XL-BFD Model<b>

In [3]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_bfd", do_lower_case=False )

In [4]:
model = TFT5Model.from_pretrained("Rostlab/prot_t5_xl_bfd")

All model checkpoint weights were used when initializing TFT5Model.

All the weights of TFT5Model were initialized from the model checkpoint at Rostlab/prot_t5_xl_bfd.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5Model for predictions without further training.


In [5]:
gc.collect()

0

<b>3. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [6]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [7]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>4. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [8]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True, return_tensors="tf")

In [9]:
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']

<b>5. Extracting sequences' features and load it into the CPU if needed<b>

In [10]:
embedding = model(input_ids, decoder_input_ids=input_ids)

In [11]:
embedding = np.asarray(embedding)

In [12]:
attention_mask = np.asarray(attention_mask)

In [13]:
# For feature extraction we recommend to use the encoder embedding
encoder_embedding = embedding[2]
decoder_embedding = embedding[0]

<b>7. Remove padding (\<pad\>) and special tokens (\</s\>) that is added by ProtT5-XL-BFD model<b>

In [18]:
features = [] 
for seq_num in range(len(encoder_embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = encoder_embedding[seq_num][:seq_len-1]
    features.append(seq_emd)

In [19]:
print(features)

[<tf.Tensor: shape=(7, 1024), dtype=float32, numpy=
array([[ 0.29069257, -0.20804922, -0.21934749, ...,  0.33846596,
         0.46031442, -0.16942428],
       [ 0.20362839, -0.12995914, -0.17139854, ...,  0.10642263,
        -0.37738237,  0.05401217],
       [ 0.03736717, -0.08506598, -0.23616521, ...,  0.25152585,
         0.1333862 ,  0.03063789],
       ...,
       [ 0.47787994, -0.23029439, -0.10682338, ...,  0.40571734,
         0.52513736,  0.19175257],
       [ 0.07990429,  0.07094713, -0.08554702, ...,  0.2494752 ,
         0.36538345, -0.45506254],
       [ 0.09110186,  0.17046821,  0.42079183, ...,  0.25626653,
         0.02010926, -0.11016779]], dtype=float32)>, <tf.Tensor: shape=(5, 1024), dtype=float32, numpy=
array([[ 0.32152936, -0.09309337, -0.15645395, ..., -0.02557738,
         0.06829377, -0.15527725],
       [ 0.10843595, -0.06435563,  0.15844898, ...,  0.05685001,
        -0.16503637,  0.15184814],
       [ 0.21142836,  0.11896314,  0.08190013, ...,  0.2332427 ,
  